In [2]:
import json
import sys
sys.path.insert(1, "../")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from parse import parser
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA

In [3]:
dataMario = pd.read_csv("../csv_downsampled/EVDb_SimCaPlus_Mario0006_downsampled_500.csv")
dataMario.head()

,FRAME,TIME,PHANTOM,MARKER_NR,TRAJECTORY_TYPE,X,Y,Z,VX,VY,VZ,VELOCITY
0,1,0.0,skull,midpoint,calculated,391.053,333.058,184.309,NaN,NaN,NaN,NaN
1,1,0.0,skull,phantom1,Measured,420.188,336.076,221.468,NaN,NaN,NaN,NaN
2,1,0.0,skull,phantom2,Measured,392.340,395.010,178.267,NaN,NaN,NaN,NaN
3,1,0.0,skull,phantom3,Measured,452.539,326.562,134.224,NaN,NaN,NaN,NaN
4,1,0.0,skull,phantom4,Measured,390.401,301.714,187.366,NaN,NaN,NaN,NaN


In [4]:
dataMario.shape

(7440, 12)

In [21]:
dataMario["ANGLEXY"] = np.nan
dataMario["ANGLEXZ"] = np.nan
dataMario["ANGLEYZ"] = np.nan
for d in dataMario.groupby("MARKER_NR"):
    
    diff = d[1].loc[:, ["X", "Y", "Z"]].shift(-1) - d[1].loc[:, ["X", "Y", "Z"]]
    
    xy = (270 - np.tan(diff.Y / diff.X) * 180 / np.pi) % 360
    xz = (270 - np.tan(diff.Z / diff.X) * 180 / np.pi) % 360
    yz = (270 - np.tan(diff.Z / diff.Y) * 180 / np.pi) % 360
    
    results = pd.DataFrame(pd.concat([xy, xz, yz], axis=1).values, columns=["ANGLEXY", "ANGLEXZ", "ANGLEYZ"], index = diff.index)
    results.loc[results.index != results.index[-1]] = results.loc[results.index != results.index[-1]].fillna(180)
    dataMario.loc[results.index, ["ANGLEXY", "ANGLEXZ", "ANGLEYZ"]] = results

dataMario.head()

c:\Users\willi\.conda\envs\myenv\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in tan
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\willi\.conda\envs\myenv\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in tan
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\willi\.conda\envs\myenv\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in tan
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\willi\.conda\envs\myenv\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in tan
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\willi\.conda\envs\myenv\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in tan
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\willi\.conda\envs\myenv\Lib\site-packages\pandas\core\arraylike.py:396: Runt

,FRAME,TIME,PHANTOM,MARKER_NR,TRAJECTORY_TYPE,X,Y,Z,VX,VY,VZ,VELOCITY,ANGLEXY,ANGLEXZ,ANGLEYZ
0,1,0.0,skull,midpoint,calculated,391.053,333.058,184.309,NaN,NaN,NaN,NaN,180.000000,180.000000,180.000000
1,1,0.0,skull,phantom1,Measured,420.188,336.076,221.468,NaN,NaN,NaN,NaN,278.535269,305.336133,260.282716
2,1,0.0,skull,phantom2,Measured,392.340,395.010,178.267,NaN,NaN,NaN,NaN,247.323310,220.856628,88.625325
3,1,0.0,skull,phantom3,Measured,452.539,326.562,134.224,NaN,NaN,NaN,NaN,153.290720,221.060996,274.885496
4,1,0.0,skull,phantom4,Measured,390.401,301.714,187.366,NaN,NaN,NaN,NaN,284.748320,243.890155,261.459441


In [22]:
dataMario

,FRAME,TIME,PHANTOM,MARKER_NR,TRAJECTORY_TYPE,X,Y,Z,VX,VY,VZ,VELOCITY,ANGLEXY,ANGLEXZ,ANGLEYZ
0,1,0.000,skull,midpoint,calculated,391.053,333.058,184.309,NaN,NaN,NaN,NaN,180.000000,180.000000,180.000000
1,1,0.000,skull,phantom1,Measured,420.188,336.076,221.468,NaN,NaN,NaN,NaN,278.535269,305.336133,260.282716
2,1,0.000,skull,phantom2,Measured,392.340,395.010,178.267,NaN,NaN,NaN,NaN,247.323310,220.856628,88.625325
3,1,0.000,skull,phantom3,Measured,452.539,326.562,134.224,NaN,NaN,NaN,NaN,153.290720,221.060996,274.885496
4,1,0.000,skull,phantom4,Measured,390.401,301.714,187.366,NaN,NaN,NaN,NaN,284.748320,243.890155,261.459441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7435,12020,60.095,catheter3_mid3,tip3,Measured,159.942,333.667,487.839,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7436,12020,60.095,catheter3_mid3,tip4,Measured,117.212,318.223,514.824,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7437,12020,60.095,catheter3_mid3,tip5,Measured,151.675,294.355,516.437,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7438,12020,60.095,catheter3_mid3,tip6,Measured,129.790,292.759,571.621,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
px.line(dataMario, x="TIME", y="ANGLEXY", color="MARKER_NR")